<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_Self_Querying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

RAG는 여러 과정을 거치면서 진행된다. 매 과정이 치명적으로 중요하다.

- 멀티쿼리 : 대충질문해도 좋은 답변을 원하는 이들을 위해 다음에 집중해야 한다.
- 페어런트 도큐먼트 : 앞뒤 문장을 잘 담아야 하고(chunk의 중요성)
- 셀프쿼리(질문재해석) : 시맨틱검색 말고 쿼리가 필요한 경우
  - 시맨틱검색이란 질문문장과 임베딩데이터의 벡터값에 따라 유사데이터를 걸러내는 것인데, 이 경우 질문의 모양이 조금만 달라져도 추출 데이터 자체가 달라진다. 이것은 '질문이 우선되는 구조'이기 때문이다.
  이 경우, 데이터를 중심으로 사용자의 질문을 참고하여 쿼리를 날려서 정리해야 할 필요가 생길 수 있다. 이것도 고려해야 한다.
- 타임 웨이티드 : 오래된 자료는 덜 참고했으면...
  - 최근에 올라간 자료에 무게를 둬서 답변을 참고했으면 좋겠다.

## Self-Querying Retriever

일반적인 시맨틱서치는 사용자의 질문에 등장하는 단어/문장과 유사한 벡터값을 가진 문서를 반환한다. 즉... 똑같은 답변이 나와야 할 질문이라도, 질문에 사용된 단어나 문장에 따라서 다른 문서가 반환될 수 있는 것. 사용자의 질문를 기준으로 돌아가는 구조라고 보면 되겠다.

하지만, 통계데이터의 경우, 질문에 따라 추출값이 달라져서는 안되며, 해당 (이를테면) 테이블 안에 질문자가 사용한 단어/단위등이 정확히 포함되어 있지 않을 가능성이 있기에, '사용자의 질문에 대답하기 위해서 데이터의 특정부분을 가공/필터링하는 과정'이 필요하게 되는데, 이 과정에서 사용되는 것이 셀프쿼리이다. 즉 주어진 데이터를 가공/종합하여 답변하는 구조이다.

## Setting

In [1]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U -q langchain pypdf sentence_transformers chromadb langchain-openai
!pip install -q langchain pypdf sentence_transformers chromadb langchain-openai

In [3]:
# 유틸들
import math
import time

def trace(func):
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f"{end - start:.5f} sec")
    return wrapper

In [4]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 265
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

In [5]:
URL = 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn'
URLs = [
    URL,
    'https://n.news.naver.com/article/079/0003862456?cds=news_media_pc&type=editn'
]
SOURCE_folder = "/content/drive/MyDrive/data/test"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

chunk_size = 200

DB_URL = '/content/drive/MyDrive/vector_upgrade/'
MODEL_EMBED= "jhgan/ko-sbert-nli"

llm = openai_compbot
llm_chat = openai_chatbot

# 코드

In [6]:
!pip install -U -q lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.2 MB/s eta 0:00:00


## ParentDocumentRetriever

In [14]:
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [8]:
# 데이터
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [11]:
ko_embed = HuggingFaceEmbeddings(
    model_name= MODEL_EMBED,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

vectorstore = Chroma.from_documents(docs, ko_embed)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [18]:
# 데이터 : 각 메타속성이 뭐하는 건지를 알려준다. 그래야 얘가 뭔지 알아먹고 이용을 할테니까.
document_content_description = 'Brief summary of a movie'
metadata_field_info = [
    # 여기 없으면 애가 답변할 수 없음. 이를테면 재패니메이션에 대해서 물어보면... 대답을 못하겠지.
    # 이건 애가 할루시네이션을 일으키지 않도록, 책임소재를 확실히 하기 위해서 필요한 과정.
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [19]:
llm = openai_chatbot

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose = True
)

In [20]:
query = 'the movies with rated higher than 8.5'
retriever.get_relevant_documents(query)

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006})]

In [ ]:
# 이제 위의 리트리버를 이용해서 로딩한 문서를 넣어준다.
# 다른 경우와는 달리, 위에서 설정했던 스플리터를 사용해서 얘가 다 알아서 잘라주고, 부모-자식간의 연결고리도 마련한다.
# 그래서 시간이 현기증날만큼 조온나 오래걸린다. GPU를 써도 오래걸린다. 와 짜증 진짜...

@trace
# def make_context():
#     retriever.add_documents(documents, ids=None)

# make_context()
# len(list(store.yield_keys()))

165.58265 sec


71

In [ ]:
# import logging

# logging.basicConfig()
# logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [ ]:
# unique_docs = retriever_from_llm.get_relevant_documents(query=question)
# len(unique_docs)

In [ ]:
# unique_docs

In [ ]:
https://www.youtube.com/watch?v=wQEl0GGxPcM&t=306